In [9]:
import os
import csv
import heapq
import numpy as np

In [10]:
data_folder = 'C:/D/coursera/discrete_opt/coloring/data'
data_files = None
for dirpath, dirnames, filenames in os.walk(data_folder):
    data_files = [os.path.join(dirpath, fname) for fname in filenames if fname[:2] == 'gc' and fname[-3:] != 'txt'\
                 and fname[-3:] != 'put']
    
filepaths = list(reversed(data_files))
print('File paths \n', filepaths)

File paths 
 ['C:/D/coursera/discrete_opt/coloring/data\\gc_70_9', 'C:/D/coursera/discrete_opt/coloring/data\\gc_70_7', 'C:/D/coursera/discrete_opt/coloring/data\\gc_70_5', 'C:/D/coursera/discrete_opt/coloring/data\\gc_70_3', 'C:/D/coursera/discrete_opt/coloring/data\\gc_70_1', 'C:/D/coursera/discrete_opt/coloring/data\\gc_50_9', 'C:/D/coursera/discrete_opt/coloring/data\\gc_50_7', 'C:/D/coursera/discrete_opt/coloring/data\\gc_50_5', 'C:/D/coursera/discrete_opt/coloring/data\\gc_50_3', 'C:/D/coursera/discrete_opt/coloring/data\\gc_50_1', 'C:/D/coursera/discrete_opt/coloring/data\\gc_500_9', 'C:/D/coursera/discrete_opt/coloring/data\\gc_500_7', 'C:/D/coursera/discrete_opt/coloring/data\\gc_500_5', 'C:/D/coursera/discrete_opt/coloring/data\\gc_500_3', 'C:/D/coursera/discrete_opt/coloring/data\\gc_500_1', 'C:/D/coursera/discrete_opt/coloring/data\\gc_4_1', 'C:/D/coursera/discrete_opt/coloring/data\\gc_250_9', 'C:/D/coursera/discrete_opt/coloring/data\\gc_250_7', 'C:/D/coursera/discrete_op

In [11]:
class Node:
    
    def __init__(self, parent = None, color= None, index = -1, degree = -1):
        
        self.color = color
        self.index = index
        self.degree = degree
        self.parent = parent
        self.child_indices = []
        self.depth = 0
        if parent:
            self.depth = parent.depth+1    
            
    def __repr__(self):
        """Use by calling repr(node)"""
        return "Node(c: " + repr(self.color) + ", idx: " + repr(self.index) + ", deg: " + repr(self.degree) + ")"    
    
    def expand(self, index):
        if index in self.child_indices:
            #print('Index already exists in the children list')
            return False
        else:
            self.child_indices.append(index)
            #print('Node %d: add child index %d' % (self.index, index))
            return True
            
    def set_color(self, color):
        self.color = color
        
    def __lt__(self, other):
        return self.degree > other.degree
    
# class NodeInfo:
    
#     def __init__(self, index=-1, degree=-1):
#         self.index = index
#         self.degree = degree
       
#     def __repr__(self):
#         """Use by calling repr(node)"""
#         return "Node info(idx: " + repr(self.index) + ", deg: " + repr(self.degree) + ")"      
        
#     def __lt__(self, other):
#         return self.degree > other.degree
    
class Graph:# undirected
    
    def __init__(self, edges=None, root=None):
        self.edges = None
        self.root = None
        self.vertices = dict()#[]# index to the list denotes the vertex index
        #self.vertex_info = []# heapq collecting index and degree
        
    def add_vertex(self, index, child):
        #print('In add_vertex, ', self.vertices ) 
        if index in self.vertices: #index < len(self.vertices):
            self.vertices[index].degree += 1
            #self.vertex_info[index].degree +=1
            #print('In add_vertex, Add degree to vertex %d' % index)
            #print('Index %d, Vertex degree %d' % (index, self.vertices[index].degree))
        else:
            parent, color, degree = None, None, 1
            self.vertices[index] = Node(parent, color, index, degree)
            #self.vertex_info.append(NodeInfo(index, degree))
            #print('Index {}, Node {}'.format(index, self.vertices[index]))
            #print('In add_vertex, Add new vertex %d' % index)
        self.vertices[index].expand(child)
        
        #print('Vertices \n', self.vertices)
        #heapq.heapify(self.vertices)
        #print('Vertex Info\n', self.vertex_info)        
            
    def set_edges(self,edges):
        self.edges = edges
        for edge in edges:
            # build a list of vertices
            self.add_vertex(edge[0], edge[1])
            self.add_vertex(edge[1], edge[0])
            
        #print('Vertices \n', self.vertices)
        #heapq.heapify(self.vertices)
        #print('Vertex Info\n', self.vertex_info)
        
    def print_graph(self):
        for index, vertex in self.vertices.items():
            rep = "Index " + str(index) + ", Node(c: " + str(vertex.color) + ", idx: " + str(vertex.index) + ", deg: " + str(vertex.degree)
            if vertex.child_indices:
                rep = rep + ", children: "+','.join([ str(child) for child in vertex.child_indices]) + ' )'
            print(rep)
                
        

In [12]:
# child have different color than parent
def print_graph(vertices):
    
    for index, vertex in vertices.items():
        rep = "Index " + str(index) + ", Node(c: " + str(vertex.color) + ", idx: " + str(vertex.index) + ", deg: " + str(vertex.degree)
        if vertex.child_indices:
            rep = rep + ", children: "+','.join([ str(child) for child in vertex.child_indices]) + ' )'
        print(rep)

def get_color(colors, lower_bound, upper_bound):
    
    color = None
    if not colors: color = 0
    else: color = max(colors)+1
    
    #if color <= lower_bound or color > upper_bound:
    if color > upper_bound:
        raise ValueError('Color is below the lower bound or over the upper bound')
        
    return color
                
def node_coloring(index, colors, vertices, lower_bound, upper_bound, color_frequency):
    
    #print('In node_coloring, vertex {}, color {}'.format(index, vertices[index].color))
    if vertices[index].color != None: 
        #print('Vertex %d already explored' % index)
        return False
        
    child_colors = set([vertices[cidx].color for cidx in vertices[index].child_indices if vertices[cidx].color != None])
    color, usable_colors = None, None
    
    usable_colors = colors.difference(child_colors)
    
    #print('\nIn vertex {}, Num children {}'.format(index, len(vertices[index].child_indices)))
    #print('Colors {}, Child colors {}, Usable color {}'.format(colors,child_colors,usable_colors))
    
    if usable_colors:    
    
        frequency, np_usable_color = [], None
        for u in usable_colors:
            frequency.append(color_frequency[u])
        sorted_indices = sorted(range(len(frequency)), key=lambda k: frequency[k], reverse=True)
        np_usable_color = np.asarray(list(usable_colors))
        np_usable_color = np_usable_color[sorted_indices]
        
        if np_usable_color.size > 0: color = np_usable_color[0] # [-1] --> 216
        else: color = usable_colors.pop()
        #print('Use existing color %d' % color)
    else: # every color in children are in colors
        color = get_color(colors, lower_bound, upper_bound)
        #print('Use new color %d' % color)
    if color in color_frequency: color_frequency[color] += 1
    else: color_frequency[color] = 0
    #print('In vertex %d, color %d' % (index, color))
            
    vertices[index].color = color
    colors.add(color)
    #print_graph(vertices)
    
    return True
    
def graph_coloring(vertices):
    
    explored = set()
    colors = set()
    color_frequency = dict()
    #heapq.heapify(vertex_info)
    vertex_info = []
    for idx, v in vertices.items():
        heapq.heappush(vertex_info, (-v.degree, v.index))
        
    #print('Original vertices\n', vertices)
    #print('Sorted vertex info\n', vertex_info)
    
    lower_bound, upper_bound = -vertex_info[0][0], len(vertex_info)
    #print('Lower c bound %d, higher bound %d ' % (lower_bound, upper_bound))
    
    while vertex_info:

        vinfo = heapq.heappop(vertex_info)
        #print('\n\n New vertex %d with degree %d' % (vinfo[1], -vinfo[0]))
        
        index = vinfo[1]
        if index in explored: continue
            
        flag = node_coloring(index, colors, vertices, lower_bound, upper_bound, color_frequency)
        if flag: explored.add(index)
            
        for cidx in vertices[index].child_indices:
            flag = node_coloring(cidx, colors, vertices, lower_bound, upper_bound, color_frequency)
            if flag: explored.add(cidx)
                
        #print('Explored node indices ', explored)
        #print('Used colors ', colors)
        #print_graph(vertices)
    
    #selected_colors = [vertices[i].color for i in range(len(vertices))]
    return len(colors), [vertices[i].color for i in range(len(vertices))], vertices, color_frequency

In [15]:
def solve_it(input_data):
    # Modify this code to run your optimization algorithm

    # parse the input
    lines = input_data.split('\n')

    first_line = lines[0].split()
    node_count = int(first_line[0])
    edge_count = int(first_line[1])
    #print('Node count %d, edge count %d' % (node_count, edge_count))

    edges = []
    for i in range(1, edge_count + 1):
        line = lines[i]
        parts = line.split()
        edges.append((int(parts[0]), int(parts[1])))
        
    #### play with graph
    graph = Graph()
    graph.set_edges(edges)
    #graph.print_graph()
    #print('---------------Start graph coloring ------------------')
    num_colors, solution, vertices, color_frequency = graph_coloring(graph.vertices)
    #print('Vertices ', vertices)
    vertex_colors = [(v.index, v.color) for k, v in vertices.items()]
    #print('Vertex color', [(v.index, v.color) for v in vertices])
    
    #######
    # build a trivial solution
    # every node has its own color

    # prepare the solution in the specified output format
    output_data = str(num_colors) + ' ' + str(0) + '\n'
    output_data += ' '.join(map(str, solution))

    return output_data#, vertex_colors, color_frequency


In [77]:
filename = os.path.join(data_folder,'gc_1000_7')
with open(filename, 'r') as input_data_file:
    input_data = input_data_file.read()
    
output, vertex_colors, color_frequency = solve_it(input_data)
print('\n\nVertex colors\n', vertex_colors)
print('\n\nColors frequence\n', color_frequency)
print('\n\n Output\n', output)

---------------Start graph coloring ------------------
In node_coloring, vertex 579, color None
In vertex 579, color 0
In node_coloring, vertex 0, color None
In vertex 0, color 1
In node_coloring, vertex 1, color None
In vertex 1, color 2
In node_coloring, vertex 2, color None
In vertex 2, color 3
In node_coloring, vertex 3, color None
In vertex 3, color 4
In node_coloring, vertex 4, color None
In vertex 4, color 1
In node_coloring, vertex 7, color None
In vertex 7, color 2
In node_coloring, vertex 8, color None
In vertex 8, color 5
In node_coloring, vertex 9, color None
In vertex 9, color 3
In node_coloring, vertex 10, color None
In vertex 10, color 6
In node_coloring, vertex 11, color None
In vertex 11, color 4
In node_coloring, vertex 12, color None
In vertex 12, color 3
In node_coloring, vertex 13, color None
In vertex 13, color 6
In node_coloring, vertex 14, color None
In vertex 14, color 7
In node_coloring, vertex 15, color None
In vertex 15, color 8
In node_coloring, vertex 16, 

In [57]:
a = np.zeros((1,), dtype=np.int32)
index = [0]
a[index]

array([0])

In [6]:
filename = os.path.join(data_folder,'gc_11_16')
with open(filename, 'r') as input_data_file:
    input_data = input_data_file.read()
    
print(solve_it(input_data))

3 0
1 0 1 2 0 1 0 0 0 1 1 1 1


In [7]:
filename = os.path.join(data_folder,'gc_4_1')
with open(filename, 'r') as input_data_file:
    input_data = input_data_file.read()
    
print(solve_it(input_data))

2 0
1 0 1 1


In [33]:
filename = os.path.join(data_folder,'gc_1000_7')
with open(filename, 'r') as input_data_file:
    input_data = input_data_file.read()
    
output, vertex_colors, color_frequency = solve_it(input_data)
print('\n\nVertex colors\n', vertex_colors)
print('\n\nColors frequence\n', color_frequency)
print('\n\n Output\n', output)

---------------Start graph coloring ------------------
In vertex 579, color 0
In vertex 0, color 1
In vertex 1, color 2
In vertex 2, color 3
In vertex 3, color 4
In vertex 4, color 1
In vertex 7, color 2
In vertex 8, color 5
In vertex 9, color 3
In vertex 10, color 6
In vertex 11, color 4
In vertex 12, color 3
In vertex 13, color 6
In vertex 14, color 7
In vertex 15, color 8
In vertex 16, color 8
In vertex 17, color 2
In vertex 18, color 9
In vertex 19, color 7
In vertex 20, color 10
In vertex 21, color 1
In vertex 22, color 9
In vertex 23, color 3
In vertex 24, color 9
In vertex 25, color 11
In vertex 26, color 8
In vertex 27, color 4
In vertex 28, color 11
In vertex 29, color 2
In vertex 32, color 10
In vertex 33, color 12
In vertex 34, color 11
In vertex 36, color 12
In vertex 37, color 13
In vertex 38, color 13
In vertex 39, color 5
In vertex 40, color 14
In vertex 45, color 15
In vertex 46, color 5
In vertex 47, color 16
In vertex 48, color 12
In vertex 49, color 7
In vertex 50, c

AttributeError: 'int' object has no attribute 'index'

In [16]:
data_folder = 'C:/D/coursera/discrete_opt/coloring/data'
data_files = None
for dirpath, dirnames, filenames in os.walk(data_folder):
    data_files = [os.path.join(dirpath, fname) for fname in filenames if fname[:2] == 'gc' and fname[-3:] != 'txt'\
                 and fname[-3:] != 'put']
filepaths = list(reversed(data_files))

logfilename = os.path.join(data_folder, 'output.txt')
with open(logfilename, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    
    for filename in filepaths:
        with open(filename, 'r') as input_data_file:
            input_data = input_data_file.read()
        print('Input filename: ', os.path.split(filename)[-1])
        output = solve_it(input_data)
        results = output.split('\n')
        writer.writerow(results[0])
        print(output)

Input filename:  gc_70_9
34 0
1 2 1 3 0 4 5 3 2 6 31 7 5 8 9 9 10 1 5 11 12 11 13 14 15 4 12 16 15 14 17 18 6 19 20 21 13 18 21 22 23 24 8 25 6 19 20 26 27 22 17 25 10 16 26 27 24 28 29 30 6 31 32 13 2 25 32 33 31 0
Input filename:  gc_70_7
23 0
15 1 1 2 2 16 3 4 0 3 5 4 6 7 5 0 5 6 20 8 7 9 7 10 16 2 10 8 11 8 18 2 9 4 21 17 1 0 10 11 3 12 13 14 12 1 8 19 3 11 15 15 0 12 13 6 16 16 5 22 17 17 14 9 18 17 13 19 9 16
Input filename:  gc_70_5
15 0
1 1 12 2 1 3 0 2 10 2 14 1 3 2 3 2 2 4 0 4 5 5 7 2 3 5 6 4 7 4 8 8 3 4 0 5 8 0 6 8 9 8 9 4 10 8 3 3 9 10 9 10 12 11 4 13 10 11 3 11 5 13 13 6 6 12 0 7 13 12
Input filename:  gc_70_3
11 0
1 1 1 2 2 0 0 1 2 4 2 6 1 1 3 2 2 2 5 0 1 5 0 0 0 5 3 4 7 3 9 2 1 3 8 5 6 0 3 0 0 3 5 0 4 2 4 7 3 4 5 4 7 3 8 9 9 1 5 2 2 5 4 6 8 5 0 4 6 10
Input filename:  gc_70_1
5 0
3 1 0 1 3 1 0 2 1 2 1 0 2 1 0 1 1 0 0 2 0 1 2 1 1 1 0 0 0 1 0 0 0 1 2 2 4 2 1 0 1 2 2 4 1 1 0 2 0 0 4 3 0 2 2 2 3 3 1 0 1 1 1 2 0 3 0 2 0 1
Input filename:  gc_50_9
25 0
1 24 2 3 4 1 2 5 2 6 7 5

In [239]:
data_folder = 'C:/D/coursera/discrete_opt/coloring/data'
data_files = None
for dirpath, dirnames, filenames in os.walk(data_folder):
    data_files = [os.path.join(dirpath, fname) for fname in filenames if fname[:2] == 'gc' and fname[-3:] != 'txt'\
                 and fname[-3:] != 'put']
    
filepaths = list(reversed(data_files))
for filename in filepaths:
    with open(filename, 'r') as input_data_file:
        input_data = input_data_file.read()
    print('Input filename: ', os.path.split(filename)[-1])
    print(solve_it(input_data))

Input filename:  gc_70_9
34 0
1 2 1 3 0 4 5 3 2 6 31 7 5 8 9 9 10 1 5 11 12 11 13 14 15 4 12 16 15 14 17 18 6 19 20 21 13 18 21 22 23 24 8 25 6 19 20 26 27 22 17 25 10 16 26 27 24 28 29 30 19 31 32 13 2 25 32 33 31 0
Input filename:  gc_70_7
23 0
20 1 1 2 2 16 3 4 0 3 5 4 6 7 5 0 5 6 18 8 7 9 8 10 17 2 10 11 12 11 20 2 9 4 8 18 1 0 10 12 3 13 14 15 13 1 11 11 3 12 21 16 0 13 16 6 14 17 17 22 20 18 15 9 7 17 19 19 9 14
Input filename:  gc_70_5
16 0
1 1 11 2 1 3 10 2 14 2 15 1 3 2 3 2 2 4 7 4 5 11 12 2 3 5 6 4 7 4 8 8 3 4 9 5 8 13 6 8 10 8 9 9 0 8 3 3 7 10 6 9 12 9 4 13 0 10 1 14 5 0 0 12 6 11 0 7 14 11
Input filename:  gc_70_3
11 0
1 1 1 2 2 7 6 1 2 4 2 7 1 1 3 2 2 2 5 0 1 9 6 8 0 5 3 4 7 3 0 2 1 3 9 8 0 8 3 6 0 8 0 9 4 2 4 9 3 4 5 4 6 3 5 0 8 1 5 8 2 5 4 0 8 5 8 4 6 10
Input filename:  gc_70_1
6 0
2 1 0 0 3 0 3 5 1 0 1 2 0 1 0 1 0 2 2 0 3 0 3 1 1 2 2 2 3 1 2 0 2 1 0 2 0 0 1 0 1 0 0 3 0 1 3 0 2 3 4 4 2 2 0 2 4 3 0 2 1 1 1 2 2 3 1 0 2 1
Input filename:  gc_50_9
25 0
1 24 2 3 4 1 2 5 2 6 

In [ ]:
def graph_coloring(vertices, vertex_info):
    
    explored = set()
    colors = set()
    heapq.heapify(vertex_info)
    print('Original vertices\n', vertices)
    print('Sorted vertex info\n', vertex_info)
    
    lower_bound, upper_bound = vertex_info[0].degree, len(vertex_info)
    print('Lower c bound %d, higher bound %d ' % (lower_bound, upper_bound))
    
    while vertex_info:
        vinfo = heapq.heappop(vertex_info)
        index = vinfo.index
        
        if index in explored: continue
            
        if not vertices[index].color:
            # select color
            color = get_color(colors, lower_bound, upper_bound)
            # set color
            colors.add(color)
            vertices[index].color = color   
            print('New color %d' % color)
            # mark vertex as explored
            explored.add(index)
            
        for child in vertices[index].child_indices:
            # check child nodes
            if not vertices[child].color:
                # select color
                color = get_color(colors, lower_bound, upper_bound)
                # set color
                vertices[child].color = color
                colors.add(color)
                print('New color to child %d' % color)
                # mark as visited
                explored.add(child) 
            else:
                print('Parent node %d, parent color %d, child index %d, child color %d' % (index, vertices[index].color,
                    child, vertices[child].color))
        print('Explored node indices ', explored)
        print('Used colors ', colors)